# Assignment 4 - Predicting and understanding viewer engagement with educational videos 

With the accelerating popularity of online educational experiences, the role of online lectures and other educational video continues to increase in scope and importance. Open access educational repositories such as <a href="http://videolectures.net/">videolectures.net</a>, as well as Massive Open Online Courses (MOOCs) on platforms like Coursera, have made access to many thousands of lectures and tutorials an accessible option for millions of people around the world. Yet this impressive volume of content has also led to a challenge in how to find, filter, and match these videos with learners. This assignment gives you an example of how machine learning can be used to address part of that challenge.

## About the prediction problem

One critical property of a video is engagement: how interesting or "engaging" it is for viewers, so that they decide to keep watching. Engagement is critical for learning, whether the instruction is coming from a video or any other source. There are many ways to define engagement with video, but one common approach is to estimate it by measuring how much of the video a user watches. If the video is not interesting and does not engage a viewer, they will typically abandon it quickly, e.g. only watch 5 or 10% of the total. 

A first step towards providing the best-matching educational content is to understand which features of educational material make it engaging for learners in general. This is where predictive modeling can be applied, via supervised machine learning. For this assignment, your task is to predict how engaging an educational video is likely to be for viewers, based on a set of features extracted from the video's transcript, audio track, hosting site, and other sources.

We chose this prediction problem for several reasons:

* It combines a variety of features derived from a rich set of resources connected to the original data;
* The manageable dataset size means the dataset and supervised models for it can be easily explored on a wide variety of computing platforms;
* Predicting popularity or engagement for a media item, especially combined with understanding which features contribute to its success with viewers, is a fun problem but also a practical representative application of machine learning in a number of business and educational sectors.


## About the dataset

We extracted training and test datasets of educational video features from the VLE Dataset put together by researcher Sahan Bulathwela at University College London. 

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single educational video, and includes information about diverse properties of the video content as described further below. The target variable is `engagement` which was defined as True if the median percentage of the video watched across all viewers was at least 30%, and False otherwise.

Note: Any extra variables that may be included in the training set are simply for your interest if you want an additional source of data for visualization, or to enable unsupervised and semi-supervised approaches. However, they are not included in the test set and thus cannot be used for prediction. **Only the data already included in your Coursera directory can be used for training the model for this assignment.**

For this final assignment, you will bring together what you've learned across all four weeks of this course, by exploring different prediction models for this new dataset. In addition, we encourage you to apply what you've learned about model selection to do hyperparameter tuning using training/validation splits of the training data, to optimize the model and further increase its performance. In addition to a basic evaluation of model accuracy, we've also provided a utility function to visualize which features are most and least contributing to the overall model performance.

**File descriptions** 
    assets/train.csv - the training set (Use only this data for training your model!)
    assets/test.csv - the test set
<br>

**Data fields**

train.csv & test.csv:

    title_word_count - the number of words in the title of the video.
    
    document_entropy - a score indicating how varied the topics are covered in the video, based on the transcript. Videos with smaller entropy scores will tend to be more cohesive and more focused on a single topic.
    
    freshness - The number of days elapsed between 01/01/1970 and the lecture published date. Videos that are more recent will have higher freshness values.
    
    easiness - A text difficulty measure applied to the transcript. A lower score indicates more complex language used by the presenter.
    
    fraction_stopword_presence - A stopword is a very common word like 'the' or 'and'. This feature computes the fraction of all words that are stopwords in the video lecture transcript.
    
    speaker_speed - The average speaking rate in words per minute of the presenter in the video.
    
    silent_period_rate - The fraction of time in the lecture video that is silence (no speaking).
    
train.csv only:
    
    engagement - Target label for training. True if learners watched a substantial portion of the video (see description), or False otherwise.
    

## Evaluation

Your predictions will be given as the probability that the corresponding video will be engaging to learners.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model with an AUC (area under ROC curve) of at least 0.8 passes this assignment, and over 0.85 will receive full points.
___

For this assignment, create a function that trains a model to predict significant learner engagement with a video using `asset/train.csv`. Using this model, return a Pandas Series object of length 2309 with the data being the probability that each corresponding video from `readonly/test.csv` will be engaging (according to a model learned from the 'engagement' label in the training set), and the video index being in the `id` field.

Example:

    id
       9240    0.401958
       9241    0.105928
       9242    0.018572
                 ...
       9243    0.208567
       9244    0.818759
       9245    0.018528
             ...
       Name: engagement, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out and check your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. 

* Try to avoid global variables. If you have other functions besides engagement_model, you should move those functions inside the scope of engagement_model.

* Be sure to first check the pinned threads in Week 4's discussion forum if you run into a problem you can't figure out.

### Extensions

* If this prediction task motivates you to explore further, you can find more details here on the original VLE dataset and others related to video engagement: https://github.com/sahanbull/VLE-Dataset



In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

np.random.seed(0)   # Do not change this value: required to be compatible with solutions generated by the autograder.

In [ ]:
train_data = pd.read_csv("assets/train.csv")
test_data = pd.read_csv("assets/test.csv")

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.drop(columns=['id'], inplace=True)
train_data.describe()

In [ ]:
test_data.describe()

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

In [ ]:
train_data['engagement'].replace({False: 0, True: 1}, inplace=True)
train_data.head()

In [ ]:
X, y = train_data.drop(columns=['engagement']), train_data['engagement']
X.head()

In [ ]:
y.head()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(11, 5))

axes = axes.flatten()

for i, feature in enumerate(X.columns):
    ax = axes[i]
    ax.boxplot(X[feature], vert=False)
    ax.set_title(feature)
    ax.grid(True)

# Adjust layout to prevent overlap of subplots
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
outlier_count = {}

print("Shape of DataFrame before removing outliers:", X.shape)
print()

for feature in X.columns:
    Q1 = X[feature].quantile(0.25)
    Q3 = X[feature].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = X[(X[feature] < lower_bound) | (X[feature] > upper_bound)][feature]
    outlier_count[feature] = len(outliers)

    # Remove outliers from the DataFrame
    X = X[(X[feature] >= lower_bound) & (X[feature] <= upper_bound)]
    y = y.loc[X.index]

for feature, count in outlier_count.items():
    print(f"Feature '{feature}' has {count} outliers.")
    
print()
print("Shape of DataFrame after removing outliers:", X.shape)
print("Shape of 'y' after removing outliers:", y.shape)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(11, 5))

axes = axes.flatten()

for i, feature in enumerate(X.columns):
    ax = axes[i]
    ax.boxplot(X[feature], vert=False)
    ax.set_title(feature)
    ax.grid(True)

# Adjust layout to prevent overlap of subplots
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

In [ ]:
# Scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train))
X_test = pd.DataFrame(scaler.fit_transform(X_test))

X_train.head()

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# clf = RandomForestClassifier(n_estimators=250, max_depth=10)
# clf.fit(X_train, y_train)

In [ ]:
# from sklearn.metrics import roc_auc_score

# y_pred = clf.predict(X_test)
# score = roc_auc_score(y_test, y_pred)
# print(f"ROC-AUC score = {score}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [100, 200], 'max_depth': [10, 15, 20], 'min_samples_leaf': [1, 2]}

clf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5, scoring='roc_auc', verbose=42)

grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
best_clf = grid_search.best_estimator_

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred = best_clf.predict_proba(X_test)[:, 1]
score = roc_auc_score(y_test, y_pred)

print('Validation AUC score:', score)

In [497]:
y_pred.shape

(1498,)

In [ ]:
def engagement_model():
    # Importing packages
    from sklearn.preprocessing import StandardScaler
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV

    # Loading data
    train_data = pd.read_csv('assets/train.csv')
    test_data = pd.read_csv('assets/test.csv')

    # EDA Preprocessing
    X_train, y_train = train_data.drop(columns=['id', 'engagement']), train_data['engagement']
    X_test = test_data.drop(columns=['id'])

    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train))
    X_test_scaled = pd.DataFrame(scaler.fit_transform(X_test))

    # Model training optimization
    clf = RandomForestClassifier(random_state=42)
    param_grid = {'n_estimators': [100, 200], 'max_depth': [10, 15, 20], 'min_samples_leaf': [1, 2]}
    grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5, scoring='roc_auc')

    grid_search.fit(X_train_scaled, y_train)

    grid_search.best_params_

    best_clf = grid_search.best_estimator_

    y_pred = best_clf.predict_proba(X_test_scaled)[:, 1]

    return pd.Series(y_pred, index=test_data['id'], name='engagement')

# engagement_model()

In [ ]:
# stu_ans = engagement_model()
# assert isinstance(stu_ans, pd.Series), "Your function should return a pd.Series. "
# assert len(stu_ans) == 2309, "Your series is of incorrect length: expected 2309 "
# assert np.issubdtype(stu_ans.index.dtype, np.integer), "Your answer pd.Series should have an index of integer type representing video id."